In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.model_selection import train_test_split

In [2]:
#Data

training_data = [
    ['Green', 3],
    ['Green', 3],
    ['Red', 1],
    ['Red', 1],
    ['Yellow', 3],
]
labels = [
    'Apple',
    'Apple',
    'Grape',
    'Grape',
    'Lemon',
]
header = ["color", "diameter"]

In [3]:
class Tree:
    def __init__(self,X,feature_names,labels):
        
        self.X = X
        self.num_of_nodes = 0# len(self.nodes)
        self.currentsplit = 0
        self.feature_names = feature_names#coloum names
        self.labels = labels#y
        self.catagories = set(labels)
        self.child = []
        self.leaf = 0
        

In [4]:
class Node:
    def __init__(self,labels,X_ids,feature_ids):
        
        self.split_result = 0
        self.split = 0 #which column / which feature id
        self.feature_names = feature_ids
        self.labels = labels
        self.child = []
        self.ids = X_ids
        self.leaf = 0 #true or false

In [5]:
def get_features(X,labels,column,ids):
    #Felosztjuk az adatokat a kiválasztott paratméter alapján pl.:szín
    #Ezután vissza adjuk a felasztott adathalmazt az index-el együtt
    
    features = [ X[i][column] for i in ids ]#ids tell which row should we use in our calculations
    features_categories = set(features)
    
    sub_labels = []
    for i in features_categories:
        
        sub_label = [labels[x] for x in range(len(features)) if features[x]==i]#get the labels corresponing to the chosen features
        sub_index = [x for x in range(len(features)) if features[x]==i]
        sub_labels.append([sub_label,sub_index,i])
        
    return sub_labels

In [6]:
def calculate_entropy(data):

    categories = set(data)
    entropy = 0;
    
    for i in categories:
        p = data.count(i)/len(data)
        entropy -=  p*math.log(p,2)

    
    return entropy

In [7]:
def calculate_information_gain(X,labels,column,ids):
    I = 0
    
    sub_labels = get_features(X,labels,column,ids) 

    for i in sub_labels:
        entropy = calculate_entropy(i[0])
        I += entropy * len(i[0])
    
    I = I/len(labels)
    
    return I

In [8]:
def find_best_split(X,labels,feature_names,ids):
    
    largest_ig = 0
    split = 0
    I = 0 
    
    for i in feature_names:

            I = calculate_information_gain(X,labels,i,ids)
            if(I>largest_ig):
                largest_ig = I
                split = i

    return split

In [9]:
def build_tree(X,feature_names,labels,ids):
    
    #Az adatokat az indexe-ik alapán kezeljük
    
    split = 0 
    sub_nodes = []
    
    
    for i in range(len(feature_names)):
        #Kiválasztjuk azt a kategóriát ami alapján felosztjuk
        split = find_best_split(X,labels,feature_names,ids)

        
    #Miután felosztottuk az adataokat egy kategória alapján azt töröljük a listából
    #hogy töbet ne tudjuk használni
    feature_names.pop(split)
    
    #A kiválasztott kategória alapján felosztjuk az adatokat majd a node-okba helyezzük őket
    node_labels = get_features(X,labels,split,ids)    
        
    for i in node_labels:
        node = Node(i[0],i[1],feature_names)
        node.split = split
        node.split_result = i[2]
        
        #Ha elofgytak a kategóriák vagy homogén a node akkor elértünk egy leaf-hez.
        if(feature_names == [] or calculate_entropy(node.labels) == 0):
            node.leaf = 1

        sub_nodes.append(node)
        
    if(feature_names == []):
        return sub_nodes
    else:
        for node in sub_nodes:
            if(node.leaf == 0):
                #Amennyiben a node nem leaf foolytajuk a felbontást
                node.child = build_tree(X,node.feature_names,node.labels,node.ids)


    return sub_nodes
    

In [10]:
def fit(X,feature_names,labels):
    
    #Meghívjuk a döntési fát felépítő rekurzív függvényt
    #Átadjuk neki az adatokat valamint az indexeiket
    
    ids = [x for x in range(len(X))]
    feature_ids = [x for x in range(len(feature_names))]
    tree = Tree(X,feature_names,labels)
    tree.child = build_tree(X,feature_ids,labels,ids)
    return tree

In [11]:
#build_tree(training_data,header,labels)
tree = fit(training_data,header,labels)

In [12]:
def predict(node,features):
    #A kategoriák alapján eljutunk a leaf-hez
    for i in range(len(features)):
        if(node.leaf == 1):
            return set(node.labels)
        else:
            for j in node.child:
                
                    if(features[j.split] == j.split_result):
                        if(j.leaf == 1):
                            return set(j.labels)
                        else:
                            return set(predict(j,features))
            
          

In [13]:
#Kiirjük hogy mik találhatók a leaf-ekben
for i in tree.child:
    if(i.leaf == 1):
        print(set(i.labels))
    else:
        for j in i.child:
            print(set(j.labels))
            

{'Grape'}
{'Lemon'}
{'Apple'}


In [14]:
print(predict(tree,['Green', 3]))

{'Apple'}
